In [4]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
from pathlib import Path

from opencage.geocoder import OpenCageGeocode
%matplotlib inline

In [5]:
# Load .env enviroment variables
load_dotenv()

True

In [6]:
# Set GeoCode API key
# YOUR CODE HERE!
geocoder_key = os.getenv('OPEN_CAGE_API')
type(geocoder_key)

str

In [7]:
# Feed the API key into the imported OpenCageGeocode
geocoder = OpenCageGeocode(geocoder_key)

In [8]:
# Bring in the Area csv file
arena_csv = Path('Raw_Data/Arena_Location.csv')
# Read csv into a pandas Dataframe
arena = pd.read_csv(arena_csv)
arena['City'] = ''
arena['State'] = ''
arena.head()

,Team Name,Arena Name,Arena Location,Seating Capacity,Opening Year,City,State
0,Atlanta Hawks,State Farm Arena,"Atlanta, Georgia","18,118",1999.0,,
1,NaN,NaN,NaN,NaN,NaN,,
2,Boston Celtics,TD Garden,"Boston, Massachusetts","18,624",1995.0,,
3,NaN,NaN,NaN,NaN,NaN,,
4,Brooklyn Nets,Barclays Center,"Brooklyn, New York","17,732",2012.0,,


In [9]:
# Drop the null values and reset the index
arena = arena.dropna()
arena.reset_index(drop=True,inplace=True)
arena.head()

,Team Name,Arena Name,Arena Location,Seating Capacity,Opening Year,City,State
0,Atlanta Hawks,State Farm Arena,"Atlanta, Georgia","18,118",1999.0,,
1,Boston Celtics,TD Garden,"Boston, Massachusetts","18,624",1995.0,,
2,Brooklyn Nets,Barclays Center,"Brooklyn, New York","17,732",2012.0,,
3,Charlotte Hornets,Spectrum Center,"Charlotte, North Carolina","19,077",2005.0,,
4,Chicago Bulls,United Center,"Chicago, Illinois","20,917",1994.0,,


In [10]:
# Breakout the Arena location to a city and a corresponding state
arena[['City','State']] = arena['Arena Location'].str.split(pat=',',expand=True)
arena.head()

,Team Name,Arena Name,Arena Location,Seating Capacity,Opening Year,City,State
0,Atlanta Hawks,State Farm Arena,"Atlanta, Georgia","18,118",1999.0,Atlanta,Georgia
1,Boston Celtics,TD Garden,"Boston, Massachusetts","18,624",1995.0,Boston,Massachusetts
2,Brooklyn Nets,Barclays Center,"Brooklyn, New York","17,732",2012.0,Brooklyn,New York
3,Charlotte Hornets,Spectrum Center,"Charlotte, North Carolina","19,077",2005.0,Charlotte,North Carolina
4,Chicago Bulls,United Center,"Chicago, Illinois","20,917",1994.0,Chicago,Illinois


In [11]:
# Iterate over each row to get the city and state and then use the API to get the corresponding coordinates
# Create empty list
list_lat = []
list_long = []

for index, row in arena.iterrows():
    
    City = row['City']
    State = row['State']
    query = str(City)+','+str(State)
    
    results = geocoder.geocode(query)
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']
    
    list_lat.append(lat)
    list_long.append(long)
    
# Create new columns in the arena dataframe to store these coordinates
arena['lat'] = list_lat
arena['long'] = list_long

In [12]:
arena.head()

,Team Name,Arena Name,Arena Location,Seating Capacity,Opening Year,City,State,lat,long
0,Atlanta Hawks,State Farm Arena,"Atlanta, Georgia","18,118",1999.0,Atlanta,Georgia,33.748992,-84.390264
1,Boston Celtics,TD Garden,"Boston, Massachusetts","18,624",1995.0,Boston,Massachusetts,42.360253,-71.058291
2,Brooklyn Nets,Barclays Center,"Brooklyn, New York","17,732",2012.0,Brooklyn,New York,40.650102,-73.949583
3,Charlotte Hornets,Spectrum Center,"Charlotte, North Carolina","19,077",2005.0,Charlotte,North Carolina,35.227209,-80.843083
4,Chicago Bulls,United Center,"Chicago, Illinois","20,917",1994.0,Chicago,Illinois,41.875562,-87.624421


In [13]:
# Save dataframe as a csv file
arena.to_csv(r'Cleansed_csv/arena_location.csv',index=False,header=True)